In [30]:
from selenium import webdriver
from time import sleep
import time
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import csv

PATH = "msedgedriver.exe"

categories = ['DIENTHOAI', 'MAYTINHBANG', 'VOBAOOPLUNG','MIENGDAN','PINSAC']

DIENTHOAI = 'https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-cat.84.1979'
MAYTINHBANG = 'https://shopee.vn/M%C3%A1y-t%C3%ADnh-b%E1%BA%A3ng-cat.84.14026'
VOBAOOPLUNG = 'https://shopee.vn/V%E1%BB%8F-Bao-%E1%BB%90p-l%C6%B0ng-cat.84.1985'
MIENGDAN = 'https://shopee.vn/Mi%E1%BA%BFng-d%C3%A1n-m%C3%A0n-h%C3%ACnh-cat.84.2802'
PINSAC = 'https://shopee.vn/Pin-s%E1%BA%A1c-d%E1%BB%B1-ph%C3%B2ng-cat.84.2817'

qpage = lambda l,n: l+f'?page={n}'


In [33]:
search_url_dict = dict(zip(categories, [DIENTHOAI,MAYTINHBANG,VOBAOOPLUNG,MIENGDAN,PINSAC]))

In [34]:
search_url_dict

{'DIENTHOAI': 'https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-cat.84.1979',
 'MAYTINHBANG': 'https://shopee.vn/M%C3%A1y-t%C3%ADnh-b%E1%BA%A3ng-cat.84.14026',
 'VOBAOOPLUNG': 'https://shopee.vn/V%E1%BB%8F-Bao-%E1%BB%90p-l%C6%B0ng-cat.84.1985',
 'MIENGDAN': 'https://shopee.vn/Mi%E1%BA%BFng-d%C3%A1n-m%C3%A0n-h%C3%ACnh-cat.84.2802',
 'PINSAC': 'https://shopee.vn/Pin-s%E1%BA%A1c-d%E1%BB%B1-ph%C3%B2ng-cat.84.2817'}

Crawl dữ liệu từ shopee theo từng category

# Search Page
=> crawl các thông tin sợ lượt về item (tên, giá, lượt bán,...) + url dẫn đến item (1 file riêng)
# Item Page
với mỗi item trong url thì crawl comment, ratings,....

In [20]:
driver = webdriver.Edge(PATH)



In [39]:
driver.get(qpage(DIENTHOAI,0))

total_height = int(driver.execute_script("return document.body.scrollHeight"))
print(total_height)
for i in range(1, total_height, 250):
    driver.execute_script("window.scrollTo(0, {});".format(i))
time.sleep(5)

5716


In [40]:
items = driver.find_elements_by_class_name("shopee-search-item-result__item")
len(items)

50

In [37]:
#main > div > div._1Bj1VS > div._2eoI9r > div.container._1EofO_ > div.SzZ8hs > div > div.row.shopee-search-item-result__items > div:nth-child(1) > div > a

In [95]:
def collect_search_page(driver, search_url_dict, output_filename ="search.csv", pages_per_category=5):
    with open(output_filename, mode='w', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(['name', 'category','n_sold', 'price', 'shop_address','image_url', 'url'])
        for category, url in search_url_dict.items():
            for i in range(pages_per_category):
                driver.get(qpage(url,0))
                total_height = int(driver.execute_script("return document.body.scrollHeight"))

                for i in range(1, total_height, 250):
                    driver.execute_script("window.scrollTo(0, {});".format(i))
                time.sleep(1)
                items = driver.find_elements_by_class_name("shopee-search-item-result__item")
                
                for i,it in enumerate(items):
                    try:
                        #Get interest attributes
                        name = it.find_element_by_css_selector('._1NoI8_').text
                        n_sold = it.find_element_by_css_selector('._18SLBt').text
                        price = it.find_element_by_css_selector('._1w9jLI._37ge-4._2ZYSiu > span._341bF0').text
                        shop_address = it.find_element_by_css_selector('._3amru2').text   
                        image_url = it.find_element_by_css_selector('img._1T9dHf.V1Fpl5').get_attribute("src")
                        url = it.find_element_by_css_selector('a').get_attribute("href")

                        #reformat some fields
                        print("-".join([name,sold,prices,shop_address]))
                        n_sold = n_sold.split(" ")[-1] #convert n_sold to numeric
                        if n_sold[-1] == 'k':
                            n_sold = float(n_sold.replace(",",".").strip("k")) * 1000
                        else:
                            n_sold = float(n_sold)
                        price = float(price.replace(".","")) #convert price to numeric

                        csv_writer.writerow([name,category, n_sold, price,shop_address, image_url, url])

                    except: 
                        print('Missing ', i)

                time.sleep(1)

In [96]:
collect_search_page(driver, search_url_dict, pages_per_category=1)

Điện thoại Xiaomi Redmi Note 9S 4GB/64GB - Hàng chính hãng - Bảo hành điện tử 18 tháng-Đã bán 419-4.890.000-TP. Hồ Chí Minh
Điện thoại OPPO A37 Fullbox Mới - 2GB/16GB - Bảo hành 12 tháng --Đã bán 419-4.890.000-TP. Hồ Chí Minh
điện thoại Samsung Galaxy S7 chính hãng / full chức năng-Đã bán 419-4.890.000-TP. Hồ Chí Minh
Điện thoại Nokia 1280 Huyền thoại chính hãng kèm pin sạc-Đã bán 419-4.890.000-TP. Hồ Chí Minh
Điện thoại Nokia 1280 Nguyên zin Mới tinh chính hãng-Đã bán 419-4.890.000-TP. Hồ Chí Minh
Điện thoại Nokia 1280 chính hãng nguyên zin Nghe gọi to-Đã bán 419-4.890.000-TP. Hồ Chí Minh
Điện thoại Nokia 1280 Huyền thoại chính hãng kèm pin sạc-Đã bán 419-4.890.000-TP. Hồ Chí Minh
Điện Thoại Nokia 6300 Siêu Bền _ Đủ Màu _ Chính hãng Nhập Khẩu-Đã bán 419-4.890.000-TP. Hồ Chí Minh
ĐIỆN THOẠI NOKIA 1202 MỚI TINH FULL SẠC PIN CHÍNH HÃNG-Đã bán 419-4.890.000-TP. Hồ Chí Minh
điện thoại Samsung Galaxy S7 Edge 2 sim ram 4G bộ nhớ 32G mới, chơi game nặng mượt-Đã bán 419-4.890.000-TP. Hồ Chí Mi